In [1]:
import numpy as np
import pandas as pd
pd.options.mode.use_inf_as_na = True
#pd.set_option('display.max_rows', 300)
#pd.set_option('display.max_columns', 100)
import matplotlib.pyplot as plt
import japanize_matplotlib
%matplotlib inline
import re
import unicodedata
def get_east_asian_width_count(text):
    count = 0
    for c in text:
        if unicodedata.east_asian_width(c) in 'FWA':
            count += 2
        else:
            count += 1
    return count
import jaconv
import MeCab
def mecab_list(text):
    word_class = []
    tagger = MeCab.Tagger()
    node = tagger.parse(text)
    word = node.split("\n")
    for i in word:
        if i == 'EOS':
            break
        else:
            wclass = i.split(",")
            try:
                word_class.append(wclass[7])
            except:
                word_class.append(wclass[6])
    mojiretsu = "".join(word_class)
    return mojiretsu

ModuleNotFoundError: No module named 'japanize_matplotlib'

In [5]:
pip install bs4

OSError: "/bin/sh" shell not found

In [338]:
顧客ファイル = r"D:\OneDrive - 合肥スリーハイ\\tajima20210727\得意先一覧表.csv"
車両ファイル = r"D:\OneDrive - 合肥スリーハイ\\tajima20210727\車両一覧表.csv"
申請書ファイル = r"D:\OneDrive - 合肥スリーハイ\\tajima20210727\データパンチ申請書_西川美装株式会社様-ロック解除.xlsm"

In [339]:
顧客データフレーム_オリジナル = pd.read_csv(顧客ファイル, dtype="str", encoding="cp932")
車両データフレーム_オリジナル = pd.read_csv(車両ファイル, dtype="str", encoding="cp932")
会社種類対応表データフレーム_オリジナル = pd.read_excel(申請書ファイル, dtype="str", sheet_name="参考資料３ 会社種類対応表")

In [340]:
顧客データフレーム = 顧客データフレーム_オリジナル.copy()
車両データフレーム = 車両データフレーム_オリジナル.copy()
会社種類対応表データフレーム = 会社種類対応表データフレーム_オリジナル.copy()

In [341]:
顧客データフレーム = 顧客データフレーム[["得意先コード", "得意先名"]][2:].reset_index(drop=True)

In [342]:
顧客データフレーム.columns = ["顧客コード", "顧客名"]

In [343]:
#車両データフレーム.columns = ["顧客名", "車台番号", "登録番号", "初度登録", "車検日付", "電話", "通称", "住所1", "型式", "型式類別", "諸費用計", "郵便", "自賠責", "重量税", "印紙代", "代行料", "リサイクル合計", "諸費用税", "顧客コード"]

In [344]:
会社種類のシリーズ = pd.concat([会社種類対応表データフレーム["略称"], 会社種類対応表データフレーム["正式名称"]], axis=0)
会社種類のリスト = []
for i in 会社種類のシリーズ:
    会社種類のリスト.append(jaconv.h2z(i,digit=True, ascii=True))
会社種類 = pd.Series(会社種類のリスト)
会社種類 = 会社種類.drop_duplicates().reset_index(drop=True)
会社種類

0          （株）
1           株）
2           （株
3            ㈱
4          （有）
        ...   
85      独立行政法人
86    地方独立行政法人
87       弁護士法人
88      行政書士法人
89       税理士法人
Length: 90, dtype: object

In [345]:
顧客データフレーム["チェック_数値_顧客コード"] = 顧客データフレーム["顧客コード"].astype("int")

In [346]:
#車両データフレーム = 車両データフレーム[~(車両データフレーム["顧客コード"] == "顧客ｺｰﾄﾞ")]

In [347]:
#車両データフレーム["チェック_数値_顧客コード"] = 車両データフレーム["顧客コード"].astype("int")

In [348]:
顧客コードの件数 = 顧客データフレーム.groupby(["顧客コード"]).count()[["顧客名"]].reset_index(drop=False)
顧客コードの件数.columns = ["顧客コード", "件数"]

In [349]:
顧客コードの重複 = 顧客コードの件数[顧客コードの件数["件数"] != 1]
顧客コードの重複

,顧客コード,件数


In [350]:
顧客データフレーム["変換_顧客コード"] = 顧客データフレーム["顧客コード"].astype("int").astype("str")

In [351]:
顧客データフレーム["チェック_様_顧客名"] = 顧客データフレーム["顧客名"].str.strip().str[-1:]
顧客データフレーム["チェック_御中_顧客名"] = 顧客データフレーム["顧客名"].str.strip().str[-2:]
顧客データフレーム["チェック_長さ_顧客名"] = 顧客データフレーム["顧客名"].str.strip().str.len()

In [352]:
顧客データフレーム[顧客データフレーム["チェック_様_顧客名"] == "様"]

,顧客コード,顧客名,チェック_数値_顧客コード,変換_顧客コード,チェック_様_顧客名,チェック_御中_顧客名,チェック_長さ_顧客名
323,000527,原織布(株) 原－朗様,527,527,様,朗様,11


In [353]:
顧客データフレーム.iat[323, 1] = "原織布(株)\u3000原－朗"

In [354]:
顧客データフレーム[顧客データフレーム["チェック_御中_顧客名"] == "御中"]

,顧客コード,顧客名,チェック_数値_顧客コード,変換_顧客コード,チェック_様_顧客名,チェック_御中_顧客名,チェック_長さ_顧客名


In [355]:
顧客データフレーム["チェック_長さ_顧客名"].max()

21

In [356]:
会社種類

0          （株）
1           株）
2           （株
3            ㈱
4          （有）
        ...   
85      独立行政法人
86    地方独立行政法人
87       弁護士法人
88      行政書士法人
89       税理士法人
Length: 90, dtype: object

In [357]:
顧客名_会社種類除去 = []
法人フラグ = []
顧客名_空白除去 = []
for i in 顧客データフレーム["顧客名"]:
    全角 = jaconv.h2z(i,digit=True, ascii=True)
    分割 = 全角.split("　")
    k = []
    for j in 分割:
        if len(j) != 0:
            k.append(j)
    結合 = "　".join(k)
    顧客名_空白除去.append(結合.replace("　", "").strip())
    for j in 会社種類:
        結合 = 結合.replace(j, "").strip()
    顧客名_会社種類除去.append(結合)
    if 全角 == 結合:
        法人フラグ.append(0)
    else:
        法人フラグ.append(1)
顧客データフレーム["顧客名_会社種類除去"] = 顧客名_会社種類除去
顧客データフレーム["法人フラグ"] = 法人フラグ
顧客データフレーム["顧客名_空白除去"] = 顧客名_会社種類除去
顧客データフレーム

,顧客コード,顧客名,チェック_数値_顧客コード,変換_顧客コード,チェック_様_顧客名,チェック_御中_顧客名,チェック_長さ_顧客名,顧客名_会社種類除去,法人フラグ,顧客名_空白除去
0,000002,社会福祉法人博光福祉会寿里苑,2,2,苑,里苑,14,博光福祉会寿里苑,1,博光福祉会寿里苑
1,000004,有限会社 トトキ,4,4,キ,トキ,8,トトキ,1,トトキ
2,000006,西川 淳,6,6,淳,淳,4,西川 淳,0,西川 淳
3,000008,西川 勉,8,8,勉,勉,4,西川 勉,0,西川 勉
4,000010,野上 耕造,10,10,造,耕造,5,野上 耕造,0,野上 耕造
...,...,...,...,...,...,...,...,...,...,...
1102,001596,黒川 軒－,1596,1596,－,軒－,5,黒川 軒－,0,黒川 軒－
1103,001597,武 恵美子（タケマサ）,1597,1597,）,サ）,11,武 恵美子（タケマサ）,0,武 恵美子（タケマサ）
1104,001598,丸大 識世［大樹工業樋口オサム］,1598,1598,］,ム］,16,丸大 識世［大樹工業樋口オサム］,1,丸大 識世［大樹工業樋口オサム］
1105,001599,高司 久美子［ハッピーケアー社員］,1599,1599,］,員］,17,高司 久美子［ハッピーケアー社員］,0,高司 久美子［ハッピーケアー社員］


In [358]:
顧客名1のリスト = []
顧客名2のリスト = []
顧客名3のリスト = []
顧客名4のリスト = []
顧客名5のリスト = []
for i in 顧客データフレーム["顧客名_会社種類除去"]:
    顧客名1のリスト.append(i.split("　")[0])
    try:
        顧客名2のリスト.append(i.split("　")[1].split("　")[0])
    except:
        顧客名2のリスト.append(None)
    try:
        顧客名3のリスト.append(i.split("　")[2].split("　")[0])
    except:
        顧客名3のリスト.append(None)
    try:
        顧客名4のリスト.append(i.split("　")[3].split("　")[0])
    except:
        顧客名4のリスト.append(None)
    try:
        顧客名5のリスト.append(i.split("　")[4].split("　")[0])
    except:
        顧客名5のリスト.append(None)
顧客データフレーム["顧客名1"] = 顧客名1のリスト
顧客データフレーム["顧客名2"] = 顧客名2のリスト
顧客データフレーム["顧客名3"] = 顧客名3のリスト
顧客データフレーム["顧客名4"] = 顧客名4のリスト
顧客データフレーム["顧客名5"] = 顧客名5のリスト

In [359]:
顧客データフレーム[~(顧客データフレーム["顧客名4"].isnull())]

,顧客コード,顧客名,チェック_数値_顧客コード,変換_顧客コード,チェック_様_顧客名,チェック_御中_顧客名,チェック_長さ_顧客名,顧客名_会社種類除去,法人フラグ,顧客名_空白除去,顧客名1,顧客名2,顧客名3,顧客名4,顧客名5


In [360]:
顧客データフレーム = 顧客データフレーム.drop(columns=["顧客名5"])

In [361]:
顧客データフレーム

,顧客コード,顧客名,チェック_数値_顧客コード,変換_顧客コード,チェック_様_顧客名,チェック_御中_顧客名,チェック_長さ_顧客名,顧客名_会社種類除去,法人フラグ,顧客名_空白除去,顧客名1,顧客名2,顧客名3,顧客名4
0,000002,社会福祉法人博光福祉会寿里苑,2,2,苑,里苑,14,博光福祉会寿里苑,1,博光福祉会寿里苑,博光福祉会寿里苑,None,None,None
1,000004,有限会社 トトキ,4,4,キ,トキ,8,トトキ,1,トトキ,トトキ,None,None,None
2,000006,西川 淳,6,6,淳,淳,4,西川 淳,0,西川 淳,西川,淳,None,None
3,000008,西川 勉,8,8,勉,勉,4,西川 勉,0,西川 勉,西川,勉,None,None
4,000010,野上 耕造,10,10,造,耕造,5,野上 耕造,0,野上 耕造,野上,耕造,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1102,001596,黒川 軒－,1596,1596,－,軒－,5,黒川 軒－,0,黒川 軒－,黒川,軒－,None,None
1103,001597,武 恵美子（タケマサ）,1597,1597,）,サ）,11,武 恵美子（タケマサ）,0,武 恵美子（タケマサ）,武,恵美子（タケマサ）,None,None
1104,001598,丸大 識世［大樹工業樋口オサム］,1598,1598,］,ム］,16,丸大 識世［大樹工業樋口オサム］,1,丸大 識世［大樹工業樋口オサム］,丸大,識世［大樹工業樋口オサム］,None,None
1105,001599,高司 久美子［ハッピーケアー社員］,1599,1599,］,員］,17,高司 久美子［ハッピーケアー社員］,0,高司 久美子［ハッピーケアー社員］,高司,久美子［ハッピーケアー社員］,None,None


In [362]:
所有者マスタファイル = r"D:\OneDrive - 合肥スリーハイ\\tajima20210727\辞書\ShoyuMst.csv"
所有者マスタデータフレーム_オリジナル = pd.read_csv(所有者マスタファイル, dtype="str", encoding="cp932")
所有者マスタデータフレーム = 所有者マスタデータフレーム_オリジナル.copy()

In [363]:
所有者マスタデータフレーム["整形_CompanyName"] = 所有者マスタデータフレーム["CompanyName"].str.replace("株式会社", " ").str.replace("有限会社", " ").str.replace("合名会社", " ").str.replace("合資会社", " ").str.replace("合同会社", " ").str.replace("一般社団法人", " ").str.replace("社団法人", " ").str.replace("有限責任中間法人", " ").str.replace("特定非営利活動法人", " ").str.replace("一般財団法人", " ").str.replace("財団法人", " ").str.replace("農事組合法人", " ").str.replace("（株）", " ").str.replace("有限責任事業組合", " ").str.replace("共同組合", " ").str.replace("振興組合連合会", " ").str.replace("協同組合連合会", " ").str.replace("協同組合", " ").str.replace("商工振興協同組合", " ").str.replace("商工組合", " ").str.replace("協業組合", " ").str.replace("農業協同組合", " ").str.replace("商業協同組合", " ").str.strip()

In [364]:
法人種類のリスト = []
for i in 所有者マスタデータフレーム["整形_CompanyName"]:
    if "会社" in i:
        法人種類のリスト.append(i)
    elif "法人" in i:
        法人種類のリスト.append(i)
    elif "（株）" in i:
        法人種類のリスト.append(i)
    elif "組合" in i:
        法人種類のリスト.append(i)
法人種類のリスト

[]

In [365]:
個人法人分類のリスト = []
for i in 所有者マスタデータフレーム.iterrows():
    if i[1]["整形_CompanyName"] == i[1]["CompanyName"]:
        個人法人分類のリスト.append("個人")
    else:
        個人法人分類のリスト.append("法人")
所有者マスタデータフレーム["個人法人分類"] = 個人法人分類のリスト

In [366]:
所有者マスタデータフレーム = 所有者マスタデータフレーム[["CompanyName", "CompanyKana"]]
所有者マスタデータフレーム["分類"] = "所有者マスタ" 
所有者マスタデータフレーム.columns = ["漢字", "カナ", "分類"]

In [367]:
所有者マスタデータフレーム.columns = ["漢字", "カナ", "分類"]

In [368]:
所有者マスタデータフレーム

,漢字,カナ,分類
0,有限会社ホンダモーター金ケ崎商会,ﾎﾝﾀﾞﾓｰﾀｰｶﾈｶﾞｻｷｼｮｳｶｲ,所有者マスタ
1,日産コニー北海道販売株式会社,ﾆｯｻﾝｺﾆｰﾎｯｶｲﾄﾞｳﾊﾝﾊﾞｲ,所有者マスタ
2,札幌スズキ株式会社,ｻｯﾎﾟﾛｽｽﾞｷ,所有者マスタ
3,北海スズキ自動車販売株式会社,ﾎｯｶｲｽｽﾞｷｼﾞﾄﾞｳｼｬﾊﾝﾊﾞｲ,所有者マスタ
4,札幌ダイハツ販売株式会社,ｻｯﾎﾟﾛﾀﾞｲﾊﾂﾊﾝﾊﾞｲ,所有者マスタ
...,...,...,...
45172,株式会社日立ビルシステムビジネスサポート,ﾋﾀﾁﾋﾞﾙｼｽﾃﾑﾋﾞｼﾞﾈｽｻﾎﾟｰﾄ,所有者マスタ
45173,三菱自動車ファイナンス株式会社,ﾐﾂﾋﾞｼｼﾞﾄﾞｳｼｬﾌｧｲﾅﾝｽ,所有者マスタ
45174,ＵＤフィナンシャルサービス株式会社,ﾕｰﾃﾞｨｰﾌｨﾅﾝｼｬﾙｻｰﾋﾞｽ,所有者マスタ
45175,日通リース＆ファイナンス株式会社,ﾆｯﾂｳﾘｰｽｱﾝﾄﾞﾌｧｲﾅﾝｽ,所有者マスタ


In [369]:
所有者マスタデータフレーム = 所有者マスタデータフレーム.groupby(["漢字"]).first().reset_index(drop=False)

In [370]:
名前辞書姓データフレーム = pd.read_excel(r"D:\OneDrive - 合肥スリーハイ\\tajima20210727\名前辞書姓データフレーム.xlsx", dtype="str")
名前辞書名データフレーム = pd.read_excel(r"D:\OneDrive - 合肥スリーハイ\\tajima20210727\名前辞書名データフレーム.xlsx", dtype="str")

In [371]:
所有者マスタカナ = pd.merge(顧客データフレーム, 所有者マスタデータフレーム, left_on="顧客名", right_on="漢字", how="left")
所有者マスタカナ = pd.merge(所有者マスタカナ, 所有者マスタデータフレーム, left_on="顧客名_空白除去", right_on="漢字", how="left", suffixes=["_所有者マスタ原本", "_所有者マスタ空白除去"])
所有者マスタカナ = pd.merge(所有者マスタカナ, 所有者マスタデータフレーム, left_on="顧客名_会社種類除去", right_on="漢字", how="left")
所有者マスタカナ

,顧客コード,顧客名,チェック_数値_顧客コード,変換_顧客コード,チェック_様_顧客名,チェック_御中_顧客名,チェック_長さ_顧客名,顧客名_会社種類除去,法人フラグ,顧客名_空白除去,...,顧客名4,漢字_所有者マスタ原本,カナ_所有者マスタ原本,分類_所有者マスタ原本,漢字_所有者マスタ空白除去,カナ_所有者マスタ空白除去,分類_所有者マスタ空白除去,漢字,カナ,分類
0,000002,社会福祉法人博光福祉会寿里苑,2,2,苑,里苑,14,博光福祉会寿里苑,1,博光福祉会寿里苑,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000004,有限会社 トトキ,4,4,キ,トキ,8,トトキ,1,トトキ,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000006,西川 淳,6,6,淳,淳,4,西川 淳,0,西川 淳,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,000008,西川 勉,8,8,勉,勉,4,西川 勉,0,西川 勉,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,000010,野上 耕造,10,10,造,耕造,5,野上 耕造,0,野上 耕造,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1102,001596,黒川 軒－,1596,1596,－,軒－,5,黒川 軒－,0,黒川 軒－,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1103,001597,武 恵美子（タケマサ）,1597,1597,）,サ）,11,武 恵美子（タケマサ）,0,武 恵美子（タケマサ）,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1104,001598,丸大 識世［大樹工業樋口オサム］,1598,1598,］,ム］,16,丸大 識世［大樹工業樋口オサム］,1,丸大 識世［大樹工業樋口オサム］,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1105,001599,高司 久美子［ハッピーケアー社員］,1599,1599,］,員］,17,高司 久美子［ハッピーケアー社員］,0,高司 久美子［ハッピーケアー社員］,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [372]:
所有者マスタカナ[(~(所有者マスタカナ["カナ_所有者マスタ原本"].isnull())) | (~(所有者マスタカナ["カナ_所有者マスタ空白除去"].isnull()))]

,顧客コード,顧客名,チェック_数値_顧客コード,変換_顧客コード,チェック_様_顧客名,チェック_御中_顧客名,チェック_長さ_顧客名,顧客名_会社種類除去,法人フラグ,顧客名_空白除去,...,顧客名4,漢字_所有者マスタ原本,カナ_所有者マスタ原本,分類_所有者マスタ原本,漢字_所有者マスタ空白除去,カナ_所有者マスタ空白除去,分類_所有者マスタ空白除去,漢字,カナ,分類
197,000355,高田 明,355,355,明,明,4,高田 明,0,高田 明,...,None,高田 明,ﾀｶﾀﾞｱｷﾗ,所有者マスタ,高田 明,ﾀｶﾀﾞｱｷﾗ,所有者マスタ,高田 明,ﾀｶﾀﾞｱｷﾗ,所有者マスタ
517,000808,山本 豊,808,808,豊,豊,4,山本 豊,0,山本 豊,...,None,NaN,NaN,NaN,山本 豊,ﾔﾏﾓﾄﾕﾀｶ,所有者マスタ,山本 豊,ﾔﾏﾓﾄﾕﾀｶ,所有者マスタ
751,001151,オリックス自動車株式会社,1151,1151,社,会社,12,オリックス自動車,1,オリックス自動車,...,None,オリックス自動車株式会社,ｵﾘｯｸｽｼﾞﾄﾞｳｼｬ,所有者マスタ,NaN,NaN,NaN,NaN,NaN,NaN
1029,001522,高橋 英嗣,1522,1522,嗣,英嗣,5,高橋 英嗣,0,高橋 英嗣,...,None,高橋 英嗣,ﾀｶﾊｼｴｲｼﾞ,所有者マスタ,高橋 英嗣,ﾀｶﾊｼｴｲｼﾞ,所有者マスタ,高橋 英嗣,ﾀｶﾊｼｴｲｼﾞ,所有者マスタ


In [373]:
名前辞書姓カナ = pd.merge(所有者マスタカナ, 名前辞書姓データフレーム, left_on="顧客名1", right_on="漢字", how="left", suffixes=["_所有者マスタ会社種類除去", "_名前辞書姓"])
名前辞書姓カナ

,顧客コード,顧客名,チェック_数値_顧客コード,変換_顧客コード,チェック_様_顧客名,チェック_御中_顧客名,チェック_長さ_顧客名,顧客名_会社種類除去,法人フラグ,顧客名_空白除去,...,分類_所有者マスタ原本,漢字_所有者マスタ空白除去,カナ_所有者マスタ空白除去,分類_所有者マスタ空白除去,漢字_所有者マスタ会社種類除去,カナ_所有者マスタ会社種類除去,分類_所有者マスタ会社種類除去,漢字_名前辞書姓,カナ_名前辞書姓,分類_名前辞書姓
0,000002,社会福祉法人博光福祉会寿里苑,2,2,苑,里苑,14,博光福祉会寿里苑,1,博光福祉会寿里苑,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,博光福祉会寿里苑,ハッコウフクシカイジュリエン,調査済_姓
1,000004,有限会社 トトキ,4,4,キ,トキ,8,トトキ,1,トトキ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,トトキ,トトキ,調査済_姓
2,000006,西川 淳,6,6,淳,淳,4,西川 淳,0,西川 淳,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,西川,ニシカワ,名前_姓
3,000008,西川 勉,8,8,勉,勉,4,西川 勉,0,西川 勉,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,西川,ニシカワ,名前_姓
4,000010,野上 耕造,10,10,造,耕造,5,野上 耕造,0,野上 耕造,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,野上,ノガミ,名前_姓
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1102,001596,黒川 軒－,1596,1596,－,軒－,5,黒川 軒－,0,黒川 軒－,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,黒川,クロカワ,名前_姓
1103,001597,武 恵美子（タケマサ）,1597,1597,）,サ）,11,武 恵美子（タケマサ）,0,武 恵美子（タケマサ）,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,武,タケ,名前_姓
1104,001598,丸大 識世［大樹工業樋口オサム］,1598,1598,］,ム］,16,丸大 識世［大樹工業樋口オサム］,1,丸大 識世［大樹工業樋口オサム］,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,丸大,マルダイ,調査済_姓
1105,001599,高司 久美子［ハッピーケアー社員］,1599,1599,］,員］,17,高司 久美子［ハッピーケアー社員］,0,高司 久美子［ハッピーケアー社員］,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,高司,タカツカサ,調査済_姓


In [374]:
名前辞書名カナ = pd.merge(名前辞書姓カナ, 名前辞書名データフレーム, left_on="顧客名2", right_on="漢字", how="left")
名前辞書名カナ

,顧客コード,顧客名,チェック_数値_顧客コード,変換_顧客コード,チェック_様_顧客名,チェック_御中_顧客名,チェック_長さ_顧客名,顧客名_会社種類除去,法人フラグ,顧客名_空白除去,...,分類_所有者マスタ空白除去,漢字_所有者マスタ会社種類除去,カナ_所有者マスタ会社種類除去,分類_所有者マスタ会社種類除去,漢字_名前辞書姓,カナ_名前辞書姓,分類_名前辞書姓,漢字,カナ,分類
0,000002,社会福祉法人博光福祉会寿里苑,2,2,苑,里苑,14,博光福祉会寿里苑,1,博光福祉会寿里苑,...,NaN,NaN,NaN,NaN,博光福祉会寿里苑,ハッコウフクシカイジュリエン,調査済_姓,NaN,NaN,NaN
1,000004,有限会社 トトキ,4,4,キ,トキ,8,トトキ,1,トトキ,...,NaN,NaN,NaN,NaN,トトキ,トトキ,調査済_姓,NaN,NaN,NaN
2,000006,西川 淳,6,6,淳,淳,4,西川 淳,0,西川 淳,...,NaN,NaN,NaN,NaN,西川,ニシカワ,名前_姓,淳,ジュン,名前_名
3,000008,西川 勉,8,8,勉,勉,4,西川 勉,0,西川 勉,...,NaN,NaN,NaN,NaN,西川,ニシカワ,名前_姓,勉,ツトム,名前_名
4,000010,野上 耕造,10,10,造,耕造,5,野上 耕造,0,野上 耕造,...,NaN,NaN,NaN,NaN,野上,ノガミ,名前_姓,耕造,コウゾウ,名前_名
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1102,001596,黒川 軒－,1596,1596,－,軒－,5,黒川 軒－,0,黒川 軒－,...,NaN,NaN,NaN,NaN,黒川,クロカワ,名前_姓,軒－,ケンイチ,調査済_名
1103,001597,武 恵美子（タケマサ）,1597,1597,）,サ）,11,武 恵美子（タケマサ）,0,武 恵美子（タケマサ）,...,NaN,NaN,NaN,NaN,武,タケ,名前_姓,恵美子（タケマサ）,エミコ（タケマサ）,調査済_名
1104,001598,丸大 識世［大樹工業樋口オサム］,1598,1598,］,ム］,16,丸大 識世［大樹工業樋口オサム］,1,丸大 識世［大樹工業樋口オサム］,...,NaN,NaN,NaN,NaN,丸大,マルダイ,調査済_姓,識世［大樹工業樋口オサム］,ツネヨ［ダイキコウギョウヒグチオサム］,調査済_名
1105,001599,高司 久美子［ハッピーケアー社員］,1599,1599,］,員］,17,高司 久美子［ハッピーケアー社員］,0,高司 久美子［ハッピーケアー社員］,...,NaN,NaN,NaN,NaN,高司,タカツカサ,調査済_姓,久美子［ハッピーケアー社員］,クミコ［ハッピーケアーシャイン］,調査済_名


In [375]:
名前辞書一般カナ = pd.merge(名前辞書名カナ, 名前辞書名データフレーム, left_on="顧客名3", right_on="漢字", how="left", suffixes=["_名前辞書名", "_名前辞書一般"])
名前辞書一般カナ

,顧客コード,顧客名,チェック_数値_顧客コード,変換_顧客コード,チェック_様_顧客名,チェック_御中_顧客名,チェック_長さ_顧客名,顧客名_会社種類除去,法人フラグ,顧客名_空白除去,...,分類_所有者マスタ会社種類除去,漢字_名前辞書姓,カナ_名前辞書姓,分類_名前辞書姓,漢字_名前辞書名,カナ_名前辞書名,分類_名前辞書名,漢字_名前辞書一般,カナ_名前辞書一般,分類_名前辞書一般
0,000002,社会福祉法人博光福祉会寿里苑,2,2,苑,里苑,14,博光福祉会寿里苑,1,博光福祉会寿里苑,...,NaN,博光福祉会寿里苑,ハッコウフクシカイジュリエン,調査済_姓,NaN,NaN,NaN,NaN,NaN,NaN
1,000004,有限会社 トトキ,4,4,キ,トキ,8,トトキ,1,トトキ,...,NaN,トトキ,トトキ,調査済_姓,NaN,NaN,NaN,NaN,NaN,NaN
2,000006,西川 淳,6,6,淳,淳,4,西川 淳,0,西川 淳,...,NaN,西川,ニシカワ,名前_姓,淳,ジュン,名前_名,NaN,NaN,NaN
3,000008,西川 勉,8,8,勉,勉,4,西川 勉,0,西川 勉,...,NaN,西川,ニシカワ,名前_姓,勉,ツトム,名前_名,NaN,NaN,NaN
4,000010,野上 耕造,10,10,造,耕造,5,野上 耕造,0,野上 耕造,...,NaN,野上,ノガミ,名前_姓,耕造,コウゾウ,名前_名,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1102,001596,黒川 軒－,1596,1596,－,軒－,5,黒川 軒－,0,黒川 軒－,...,NaN,黒川,クロカワ,名前_姓,軒－,ケンイチ,調査済_名,NaN,NaN,NaN
1103,001597,武 恵美子（タケマサ）,1597,1597,）,サ）,11,武 恵美子（タケマサ）,0,武 恵美子（タケマサ）,...,NaN,武,タケ,名前_姓,恵美子（タケマサ）,エミコ（タケマサ）,調査済_名,NaN,NaN,NaN
1104,001598,丸大 識世［大樹工業樋口オサム］,1598,1598,］,ム］,16,丸大 識世［大樹工業樋口オサム］,1,丸大 識世［大樹工業樋口オサム］,...,NaN,丸大,マルダイ,調査済_姓,識世［大樹工業樋口オサム］,ツネヨ［ダイキコウギョウヒグチオサム］,調査済_名,NaN,NaN,NaN
1105,001599,高司 久美子［ハッピーケアー社員］,1599,1599,］,員］,17,高司 久美子［ハッピーケアー社員］,0,高司 久美子［ハッピーケアー社員］,...,NaN,高司,タカツカサ,調査済_姓,久美子［ハッピーケアー社員］,クミコ［ハッピーケアーシャイン］,調査済_名,NaN,NaN,NaN


In [376]:
顧客名カナ = []
for i in 名前辞書一般カナ["カナ_名前辞書姓"].fillna("") + " " + 名前辞書一般カナ["カナ_名前辞書名"].fillna("") + " " + 名前辞書一般カナ["カナ_名前辞書一般"].fillna(""):
    顧客名カナ.append(jaconv.z2h(i,digit=True, ascii=True).strip())
名前辞書一般カナ["顧客名カナ"] = 顧客名カナ

In [377]:
名前辞書一般カナ.to_excel(r"D:\OneDrive - 合肥スリーハイ\\tajima20210727\カナ変換.xlsx", index=False)